# Import the required libraries

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import convolve
from scipy import misc
import math

## Define Process Pipeline

### Define Image Process

In [ ]:
class BaseImageProcess:
    """
    BaseImageProcess: A base class for image processing algorithms.

    This class provides a basic framework for implementing image processing algorithms and is intended to be subclassed.
    Subclasses should implement the `apply` method to perform specific image processing operations on an input image.
    """
    def apply(self, img):
        """
        Placeholder for applying an image processing algorithm.

        Args:
            img: The input image to process.

        Returns:
            The processed image.
        """
        pass

class MedianBlur(BaseImageProcess):
    """
    MedianBlur: Applies median blur filtering to an image.

    This class extends BaseImageProcess and provides an implementation of median blur filtering, which is effective
    at removing salt-and-pepper noise.

    Attributes:
        kernel_size (int): Size of the kernel used for the median filter.
    """
    def __init__(self, kernel_size=5):
        self.kernel_size = kernel_size

    def apply(self, img):
        return cv.medianBlur(img, self.kernel_size)

class GaussianBlur(BaseImageProcess):
    """
    GaussianBlur: Applies Gaussian blur filtering to an image.

    This class provides an implementation of Gaussian blur filtering, commonly used to reduce image noise and detail.

    Attributes:
        kernel_size (int): Size of the kernel used for the Gaussian filter.
    """
    def __init__(self, kernel_size=5):
        self.kernel_size = kernel_size

    def apply(self, img):
        return cv.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0)

class BinaryThresh(BaseImageProcess):
    """
    BinaryThresh: Applies binary thresholding to an image.

    Binary thresholding converts an image to binary (black and white) based on a threshold value. Pixels above the
    threshold are set to the maximum value, and those below are set to zero.

    Attributes:
        thresh (int): Threshold value.
        max_val (int): Maximum value to use with the threshold.
    """
    def __init__(self, thresh=127, max_val=255):
        self.thresh = thresh
        self.max_val = max_val

    def apply(self, img):
        _, _img = cv.threshold(img, self.thresh, self.max_val, cv.THRESH_BINARY)
        return _img

class AdaptiveMeanThresh(BaseImageProcess):
    """
    AdaptiveMeanThresh: Applies adaptive mean thresholding to an image.

    Unlike simple thresholding, adaptive thresholding changes the threshold dynamically over the image based on local
    image characteristics.

    Attributes:
        block_size (int): Size of a pixel neighborhood used to calculate the threshold.
        c (int): Constant subtracted from the calculated mean or weighted mean.
    """
    def __init__(self, block_size=11, c=2):
        self.block_size = block_size
        self.c = c

    def apply(self, img):
        return cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, self.block_size, self.c)

class AdaptiveGaussThresh(BaseImageProcess):
    """
    AdaptiveGaussThresh: Applies adaptive Gaussian thresholding to an image.

    This method uses a weighted sum of neighbourhood values where weights are a Gaussian window, which provides
    a more natural thresholding, especially under varying illumination.

    Attributes:
        block_size (int): Size of a pixel neighborhood used to calculate the threshold.
        c (int): Constant subtracted from the calculated weighted sum.
    """
    def __init__(self, block_size=11, c=2):
        self.block_size = block_size
        self.c = c

    def apply(self, img):
        return cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, self.block_size, self.c)

class OtsuThresh(BaseImageProcess):
    """
    OtsuThresh: Applies Otsu's thresholding to automatically perform histogram shape-based image thresholding.

    This method is useful when the image contains two prominent pixel intensities and calculates an optimal threshold
    separating these two classes so that their combined spread (intra-class variance) is minimal.
    """
    def apply(self, img):
        _, _img = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        return _img

class MorphDilate(BaseImageProcess):
    """
    MorphDilate: Applies morphological dilation to an image.

    Dilation increases the white region in the image or size of the foreground object. Commonly used to accentuate
    features.

    Attributes:
        kernel_size (int): Size of the structuring element.
        iterations (int): Number of times dilation is applied.
    """
    def __init__(self, kernel_size=3, iterations=2):
        self.kernel_size = kernel_size
        self.iterations = iterations
        self.kernel = np.ones((self.kernel_size, self.kernel_size), np.uint8)

    def apply(self, img):
        return cv.dilate(img, self.kernel, iterations=self.iterations)

class MorphErode(BaseImageProcess):
    """
    MorphErode: Applies morphological erosion to an image.

    Erosion erodes away the boundaries of the foreground object and is used to diminish the features of an image.

    Attributes:
        kernel_size (int): Size of the structuring element.
        iterations (int): Number of times erosion is applied.
    """
    def __init__(self, kernel_size=3, iterations=2):
        self.kernel_size = kernel_size
        self.iterations = iterations
        self.kernel = np.ones((self.kernel_size, self.kernel_size), np.uint8)

    def apply(self, img):
        return cv.erode(img, self.kernel, iterations=self.iterations)

class LoG(BaseImageProcess):
    """
    LoG: Applies Laplacian of Gaussian filtering to an image.

    This method is used to highlight regions of rapid intensity change and is therefore often used for edge detection.
    First, it applies a Gaussian blur, then computes the Laplacian of the result.

    Attributes:
        sigma (float): Standard deviation of the Gaussian filter.
        size (int): Size of the filter kernel.
    """
    def __init__(self, sigma=2.0, size=None):
        self.sigma = sigma
        self.size = size if size is not None else int(6 * self.sigma + 1) if self.sigma >= 1 else 7
        if self.size % 2 == 0:
            self.size += 1

    def apply(self, img):
        x, y = np.meshgrid(np.arange(-self.size//2+1, self.size//2+1), np.arange(-self.size//2+1, self.size//2+1))
        kernel = -(1/(np.pi * self.sigma**4)) * (1 - ((x**2 + y**2) / (2 * self.sigma**2))) * np.exp(-(x**2 + y**2) / (2 * self.sigma**2))
        kernel = kernel / np.sum(np.abs(kernel))
        return cv.filter2D(img, -1, kernel)


class LoGConv(BaseImageProcess):
    """
    LoGConv: Implements convolution with a Laplacian of Gaussian kernel to an image.

    Similar to the LoG class, but tailored for applying custom convolution operations directly with a manually
    crafted LoG kernel.

    Attributes:
        sigma (float): Standard deviation of the Gaussian filter.
        size (int): Size of the filter kernel.
    """
    def __init__(self, sigma=2.0, size=None):
        self.sigma = sigma
        self.size = size if size is not None else int(6 * sigma + 1)
        if self.size % 2 == 0:
            self.size += 1

    def apply(self, img):
        x, y = np.meshgrid(np.arange(-self.size//2 + 1, self.size//2 + 1),
                           np.arange(-self.size//2 + 1, self.size//2 + 1))
        kernel = -(1/(np.pi * self.sigma**4)) * (1 - ((x**2 + y**2) / (2 * self.sigma**2))) * np.exp(-(x**2 + y**2) / (2 * self.sigma**2))
        kernel = kernel / np.sum(np.abs(kernel))
        return convolve(img, kernel)


## Define Pipeline Class

In [ ]:
class Pipe:
    """
    Pipe: A class to manage and execute a pipeline of image processing operations on an image.

    This class allows for the sequential application of multiple image processing algorithms, represented by instances
    of classes derived from BaseImageProcess. It manages the image processing operations as a list of tasks which are
    applied in the order they are added.

    Attributes:
        img (array-like): The base image on which the image processing operations will be applied.
        process_list (list): A list of image processing tasks (instances of BaseImageProcess).

    Methods:
        add(image_process): Adds a single image processing task to the pipeline.
        add_list(image_process_list): Adds a list of image processing tasks to the pipeline.
        clear(): Clears all image processing tasks from the pipeline.
        run(): Applies all the image processing tasks in the pipeline to the base image and displays the result.
    """
    def __init__(self):
        """
        Initializes a new Pipe instance with a base image.

        Args:
            base_img (array-like): The base image to which the image processing operations will be applied.
        """
        self.process_list = []

    def add(self, image_process : BaseImageProcess):
        """
        Adds a single image processing task to the process list.

        Args:
            image_process (BaseImageProcess): An instance of BaseImageProcess to be added to the pipeline.
        """
        self.process_list.append(image_process)

    def add_list(self, image_process_list):
        """
        Extends the process list by adding multiple image processing tasks.

        Args:
            image_process_list (list of BaseImageProcess): A list of image processing tasks to be added to the pipeline.
        """
        self.process_list.extend(image_process_list)

    def clear(self):
        """
        Clears all image processing tasks from the pipeline.

        This method resets the process list to an empty list, removing all previously added image processing tasks.
        """
        self.process_list = []

    def run(self, img):
        """
        Executes the pipeline of image processing tasks on the base image.

        This method applies each image processing task in the order they were added to the base image and displays
        each processed image step by step in a grid layout using matplotlib. Each intermediate image and the final
        processed image are displayed in grayscale.
        """
        _img = img
        num_steps = len(self.process_list) + 1
        cols = 4
        rows = math.ceil(num_steps / cols)

        plt.figure(figsize=(20, 4 * rows))

        for i, img_process in enumerate(self.process_list):
            _img = img_process.apply(_img)
            plt.subplot(rows, cols, i + 1)
            plt.imshow(_img, 'gray')
            plt.title(f"Step {i + 1}: {img_process.__class__.__name__}")
            plt.axis('off')

        plt.subplot(rows, cols, num_steps)
        plt.imshow(_img, 'gray')
        plt.title("Final Processed Image")
        plt.axis('off')
        plt.tight_layout()
        plt.show()


### Run Pipeline on Image

In [ ]:
def read_image(file_path, mode=cv.IMREAD_GRAYSCALE):
    """
    Read an image from the specified file path in grayscale.
    """
    return cv.imread(file_path, mode)

def configure_pipeline(processes):
    """
    Configure and return a processing pipeline with predefined image processing steps.
    """
    pipeline = Pipe()
    pipeline.add_list(processes)
    return pipeline

In [ ]:
IMAGE_PATH = '/content/drive/MyDrive/datasets/sat_imgs/653_2020-12-21_S2L1C_21JYJ_TCI.png'
img = read_image(IMAGE_PATH)

## Mixed Experiments

In [ ]:
processes = [
        GaussianBlur(kernel_size=5),
        LoG(sigma=2.0),
        BinaryThresh(thresh=127, max_val=255)
]

configure_pipeline(processes).run(img)

In [ ]:
processes = [
        MedianBlur(kernel_size=5),
        LoGConv(sigma=2.0),
        MorphDilate(kernel_size=3, iterations=2)
]

configure_pipeline(processes).run(img)

In [ ]:
processes = [
        GaussianBlur(kernel_size=5),
        AdaptiveGaussThresh(block_size=11, c=2),
        MorphErode(kernel_size=3, iterations=2)
]

configure_pipeline(processes).run(img)

In [ ]:
processes = [
        LoGConv(sigma=2.0),
        LoG(sigma=2.0),
        OtsuThresh(),
]
configure_pipeline(processes).run(img)

In [ ]:
processes = [
    GaussianBlur(kernel_size=7),
    OtsuThresh(),
    AdaptiveGaussThresh(block_size=11, c=1),
]

configure_pipeline(processes).run(img)

In [ ]:
processes = [
    MedianBlur(kernel_size=5),
    LoGConv(sigma=2.0),
    GaussianBlur(kernel_size=5),
    OtsuThresh(),
]

configure_pipeline(processes).run(img)

In [ ]:
processes = [
    MedianBlur(kernel_size=3),
    GaussianBlur(kernel_size=5),
    LoGConv(sigma=1.5),
    AdaptiveMeanThresh(block_size=15, c=3),
    MorphDilate(kernel_size=3, iterations=1)
]



configure_pipeline(processes).run(img)